In [1]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df=df.dropna()

In [3]:
X=df.drop('label',axis=1)

In [4]:
y=df['label']

In [5]:

y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [7]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [9]:
voc_size = 5000

#### One Hot Representation

In [11]:
messages = X.copy()

In [12]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
## Data Preprocessing
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[511, 2573, 847, 2660, 2152, 3001, 123, 1345, 1323, 937],
 [1220, 2946, 2008, 4924, 1280, 4114, 2564],
 [4905, 1105, 1418, 4660],
 [3233, 1531, 3496, 4898, 758, 4400],
 [2663, 1280, 2868, 3739, 317, 4771, 1280, 599, 3334, 4802],
 [3360,
  476,
  838,
  1008,
  1345,
  4674,
  3893,
  745,
  3103,
  2815,
  3224,
  3723,
  2211,
  577,
  2564],
 [3536, 3168, 2650, 3750, 2264, 3012, 4103, 4608, 2064, 2172, 3821],
 [1684, 3854, 2352, 4781, 4900, 3400, 4674, 2806, 2064, 2172, 3821],
 [2240, 4905, 1082, 1509, 880, 3519, 1820, 3100, 4674, 1889],
 [4336, 2406, 3629, 2744, 2045, 2690, 1259, 1657],
 [1908, 1072, 3452, 1335, 474, 2861, 4354, 4435, 2248, 2344, 2785],
 [4898, 3358, 2152, 3519, 4674, 4900],
 [4999, 4174, 820, 3967, 1587, 841, 4030, 1284, 623],
 [3914, 1902, 3393, 2072, 1810, 1271, 4461, 2064, 2172, 3821],
 [3394, 2475, 1946, 4159, 2435, 2064, 2172, 3821],
 [1237, 4778, 1306, 707, 1019, 3527, 1045, 4182, 2478, 4739],
 [653, 3020, 2946],
 [3054, 249, 1330, 1492, 4674, 3468, 2638, 25

In [20]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [21]:
onehot_repr[0]

[511, 2573, 847, 2660, 2152, 3001, 123, 1345, 1323, 937]

#### Embedding Representation

In [22]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1345 1323  937]
 [   0    0    0 ... 1280 4114 2564]
 [   0    0    0 ... 1105 1418 4660]
 ...
 [   0    0    0 ... 2064 2172 3821]
 [   0    0    0 ... 1395 4607 3778]
 [   0    0    0 ... 4332  614 3636]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  511,
       2573,  847, 2660, 2152, 3001,  123, 1345, 1323,  937])

In [33]:
## Creating model
embedding_vector_features = 40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features))
model1.build(input_shape=(None, 20))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [34]:
len(embedded_docs), y.shape

(18285, (18285,))

In [35]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [36]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### Model Training

In [38]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.7743 - loss: 0.4142 - val_accuracy: 0.9181 - val_loss: 0.1943
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9500 - loss: 0.1347 - val_accuracy: 0.9178 - val_loss: 0.2055
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9659 - loss: 0.0935 - val_accuracy: 0.9153 - val_loss: 0.2394
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9814 - loss: 0.0609 - val_accuracy: 0.9165 - val_loss: 0.2593
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9874 - loss: 0.0440 - val_accuracy: 0.9114 - val_loss: 0.2851
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9897 - loss: 0.0340 - val_accuracy: 0.9110 - val_loss: 0.3465
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9916 - loss: 0.0277 - val_accuracy: 0.9049 - val_loss: 0.3539
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9952 - loss: 0.0177 - val_accu

#### Performance Metrics and Accuracy

In [39]:
y_pred = model1.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [40]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3102,  317],
       [ 226, 2390]], dtype=int64)

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9100248550124275

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

